# Get Domain

**Author:** Andrew Loeppky (Lots of code stolen from Jamie Byer)

**Project:** Land-surface-atmosphere coupling - CMIP6 intercomparison 

This notebook is meant to acquire a dataset from the CMIP6 data library, chop out a pre-specified spatial slice (between coordinates specified by user), and save the dataset in Zarr format. Also adds a 3d pressure field variable, converting from surface pressure and sigma values $ap$ and $b$

## Helpful Docs

https://docs.google.com/document/d/1yUx6jr9EdedCOLd--CPdTfGDwEwzPpCF6p1jRmqx-0Q/edit#

https://towardsdatascience.com/a-quick-introduction-to-cmip6-e017127a49d3

https://pcmdi.llnl.gov/CMIP6/Guide/dataUsers.html

http://proj.badc.rl.ac.uk/svn/exarch/CMIP6dreq/tags/latest/dreqPy/docs/CMIP6_MIP_tables.xlsx

https://esgf-node.llnl.gov/search/cmip6/

In [1]:
# Attributes of the model we want to analyze (put in csv later)
source_id = 'GFDL-ESM4'
experiment_id = 'piControl'
table_id = 'Amon'

# Domain we wish to study
lats = (10, 20) # lat min, lat max
lons = (20, 29) # lon min, lon max
ceil = 500 # top of domain, hPa

# variables of interest
fields_of_interest = ("ps", "cl", "ta") 

In [2]:
import xarray as xr
import pooch
import pandas as pd
import fsspec
from pathlib import Path
import time
import numpy as np
import json

In [3]:
#get esm datastore
odie = pooch.create(
    path="./.cache",
    base_url="https://storage.googleapis.com/cmip6/",
    registry={
        "pangeo-cmip6.csv": None
    },
)
file_path = odie.fetch("pangeo-cmip6.csv")
df_og = pd.read_csv(file_path)

In [4]:
def fetch_var_exact(the_dict,df_og):
    the_keys = list(the_dict.keys())
    #print(the_keys)
    key0 = the_keys[0]
    #print(key0)
    #print(the_dict[key0])
    hit0 = df_og[key0] == the_dict[key0]
    if len(the_keys) > 1:
        hitnew = hit0
        for key in the_keys[1:]:
            hit = df_og[key] == the_dict[key]
            hitnew = np.logical_and(hitnew,hit)
            #print("total hits: ",np.sum(hitnew))
    else:
        hitnew = hit0
    df_result = df_og[hitnew]
    return df_result

In [5]:
def get_field(variable_id, 
              df,
              source_id=source_id,
              experiment_id=experiment_id,
              table_id=table_id):
    """
    extracts a single variable field from the model
    """

    var_dict = dict(source_id = 'GFDL-ESM4', variable_id = variable_id,
                    experiment_id = 'piControl', table_id = 'Amon')
    local_var = fetch_var_exact(var_dict, df)
    zstore_url = local_var['zstore'].array[0]
    the_mapper=fsspec.get_mapper(zstore_url)
    local_var = xr.open_zarr(the_mapper, consolidated=True)
    return local_var


In [6]:
def trim_field(df, lat, lon):
    """
    cuts out a specified domain from an xarrray field
    
    lat = (minlat, maxlat)
    lon = (minlon, maxlon)
    """
    new_field = df.sel(lat=slice(lat[0],lat[1]), lon=slice(lon[0],lon[1]))
    return new_field

## Create one big dataset to represent our domain

In [7]:
# grab all fields of interest and combine
my_fields = [get_field(field, df_og) for field in fields_of_interest]
small_fields = [trim_field(field, lats, lons) for field in my_fields]
my_ds = xr.combine_by_coords(small_fields, compat="broadcast_equals", combine_attrs="drop_conflicts")

In [8]:
# add pressure field, convert from sigma pressure
my_ds["p"] = my_ds.ap + my_ds.b * my_ds.ps
my_ds

<xarray.Dataset>
Dimensions:    (lev: 49, bnds: 2, time: 6000, lat: 10, lon: 7, plev: 19)
Coordinates:
    ap_bnds    (lev, bnds) float64 dask.array<chunksize=(49, 2), meta=np.ndarray>
    b_bnds     (lev, bnds) float64 dask.array<chunksize=(49, 2), meta=np.ndarray>
  * bnds       (bnds) float64 1.0 2.0
  * lat        (lat) float64 10.5 11.5 12.5 13.5 14.5 15.5 16.5 17.5 18.5 19.5
    lat_bnds   (lat, bnds) float64 10.0 11.0 11.0 12.0 ... 18.0 19.0 19.0 20.0
  * lev        (lev) float64 0.998 0.9928 0.9854 ... 3.934e-05 1.849e-05
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(49, 2), meta=np.ndarray>
  * lon        (lon) float64 20.62 21.88 23.12 24.38 25.62 26.88 28.12
    lon_bnds   (lon, bnds) float64 20.0 21.25 21.25 22.5 ... 27.5 27.5 28.75
  * time       (time) object 0001-01-16 12:00:00 ... 0500-12-16 12:00:00
    time_bnds  (time, bnds) object 0001-01-01 00:00:00 ... 0501-01-01 00:00:00
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
Data variables:
    ap         (lev) float64 dask.array<chunksize=(49,), meta=np.ndarray>
    b          (lev) float64 dask.array<chunksize=(49,), meta=np.ndarray>
    cl         (time, lev, lat, lon) float32 dask.array<chunksize=(12, 49, 10, 7), meta=np.ndarray>
    ps         (time, lat, lon) float32 dask.array<chunksize=(600, 10, 7), meta=np.ndarray>
    ta         (time, plev, lat, lon) float32 dask.array<chunksize=(30, 19, 10, 7), meta=np.ndarray>
    p          (lev, time, lat, lon) float64 dask.array<chunksize=(49, 600, 10, 7), meta=np.ndarray>
Attributes: (12/43)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            CMIP
    branch_method:          Coupled climate state after 400 years of spinup, ...
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    comment:                <null ref>
    ...                     ...
    sub_experiment_id:      none
    table_id:               Amon
    title:                  NOAA GFDL GFDL-ESM4 model output prepared for CMI...
    variant_info:           N/A
    variant_label:          r1i1p1f1
    version_id:             v20180701

In [9]:
## get data from desired model run

## extract desired fields with jamies functions and cut to size

## generate additional fields (like pressure levels)